<a href="https://colab.research.google.com/github/komalgupta23/Automobile_Dashboard_App/blob/main/ml_workshop_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from joblib import load, dump
from sklearn.datasets import load_wine
import plotly.express as px

In [7]:
wine = load_wine(as_frame = True)
x, y = wine.data, wine.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2 , random_state=42)
print(f"x train shape = {x_train.shape}")
print(f"x test shape = {x_test.shape}")

x train shape = (142, 13)
x test shape = (36, 13)


In [9]:
# minor setting
np.set_printoptions(precision=2)
px.defaults.template = "plotly_dark"

create pipeline

In [10]:
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(random_state=42)
)
pipeline.fit(x_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [11]:
y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)
training_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"training accuracy = {training_accuracy}")
print(f"test accuracy = {test_accuracy}")

training accuracy = 1.0
test accuracy = 1.0


In [13]:
# prompt: cross validation

cv_scores = cross_val_score(pipeline, x, y, cv=5)
print(f"cv scores = {cv_scores}")
print(f"mean score = {cv_scores.mean():.2f}")
print(f"+/- = {cv_scores.std():.2f}")

cv scores = [0.97 0.94 0.97 1.   1.  ]
mean score = 0.98
+/- = 0.02


In [15]:
feature_importance = pipeline.named_steps['randomforestclassifier'].feature_importances_
features = x.columns
indices = np.argsort(feature_importance)
#bar graph
fig = px.bar(
    x= feature_importance[indices],
    y=features[indices],
    orientation='h',
    title='Feature Importance'
)
fig.show()
#pie chart
fig = px.pie(
    values=feature_importance[indices],
    names=features[indices],
    title='Feature Importance'
)
fig.show()

In [16]:
dump(pipeline, 'model.joblib')

['model.joblib']

In [17]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.6 M

In [20]:
import gradio as gr
def predict(*features):
    model = load('model.joblib')
    cols = x.columns
    features = pd.DataFrame([features],columns = cols)
    result = model.predict(features)
    return wine.target_names[result[0]]

ui = gr.Interface(
    fn= predict,
    inputs = [
        gr.Number(label = feature) for feature in x.columns
    ],
    outputs = gr.Textbox(label = 'Wine Type'),
    examples = [
        x.iloc[0].tolist(),
        x.iloc[100].tolist(),
        x.iloc[160].tolist(),
    ]
)
ui.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9c757cafe944c606ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
